In [7]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### FIX ME #####
# Change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username, password, and CRUD Python module name
username = "root"
password = "jT0VmTn4mn"
host = "nv-desktop-services.apporto.com"
port = 31432
db = "AAC"
collection = "animals"

# Initialize AnimalShelter with correct parameters
shelter = AnimalShelter(username, password, host, port, db, collection)

# Retrieve all records from MongoDB
df = pd.DataFrame.from_records(shelter.read({}))

# Remove the '_id' column to prevent issues with ObjectID in Dash
df.drop(columns=['_id'], inplace=True)

## Debugging: Uncomment these lines if needed
# print(len(df.to_dict(orient='records')))
# print(df.columns)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'  # Replace with your image file path
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Place the HTML image tag into the layout
app.layout = html.Div([
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Center(html.B(html.H1('CS-340 Dashboard - Yancarlo Guzman'))),
    html.Hr(),
    html.Div([

        # Add code for the interactive filtering options (radio items)
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset',
            labelStyle={'display': 'inline-block'}
        )
    ]),
    html.Hr(),

    # Data Table to display MongoDB records
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,  # Display 10 rows per page
        style_table={'overflowX': 'auto'},  # Allow horizontal scrolling
        row_selectable='single',  # Allow single-row selection for map updates
        selected_rows=[0],  # Select first row by default
        style_data_conditional=[{
            'if': {'row_index': 'odd'},
            'backgroundColor': 'rgb(248, 248, 248)'
        }]
    ),
    html.Br(),
    html.Hr(),

    # This sets up the dashboard so that the chart and geolocation chart are side-by-side
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback to filter the data based on the radio button selection
@app.callback(Output('datatable-id', 'data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    # FIX ME: Add MongoDB queries based on filter type
    if filter_type == 'water':
        query = {"animal_type": "Dog", "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}, "sex_upon_outcome": "Intact Female"}
    elif filter_type == 'mountain':
        query = {"animal_type": "Dog", "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}, "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}, "sex_upon_outcome": "Intact Male"}
    elif filter_type == 'disaster':
        query = {"animal_type": "Dog", "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}, "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}, "sex_upon_outcome": "Intact Male"}
    else:
        query = {}  # Reset option returns all records

    # Retrieve filtered data
    df_filtered = pd.DataFrame.from_records(shelter.read(query))
    df_filtered.drop(columns=['_id'], inplace=True)  # Remove '_id' column for Dash
    return df_filtered.to_dict('records')

# Callback to update the chart based on filtered data
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    # FIX ME: Add code for a pie chart displaying breed distribution
    dff = pd.DataFrame.from_dict(viewData)
    if not dff.empty:
        return [
            dcc.Graph(
                figure=px.pie(dff, names='breed', title='Preferred Breeds for Rescue')
            )
        ]
    return "No data available."

# Callback to update the map based on selected data
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, selected_rows):
    if selected_rows is None:
        row = 0
    else:
        row = selected_rows[0]

    # Convert viewData (a dictionary) to a DataFrame for easy access
    dff = pd.DataFrame.from_dict(viewData)

    # If the dataframe is empty, return nothing
    if dff.empty:
        return "No data available."

    # Create a map centered at Austin, TX
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),

            # Add marker at the animal's location (assuming lat and long are in columns 13 and 14)
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                      children=[
                          dl.Tooltip(dff.iloc[row, 4]),  # Tooltip shows the breed
                          dl.Popup([
                              html.H1("Animal Name"),
                              html.P(dff.iloc[row, 9])  # Popup shows the animal's name
                          ])
                      ])
        ])
    ]

# This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []
    return [{
        'if': {'column_id': i},
        'backgroundColor': '#D2F3FF'
    } for i in selected_columns]

# Run the app inside Jupyter notebook
app.run_server(debug=True)


Dash app running on http://127.0.0.1:10207/
